## Moduly

### 1. Přesuňte třídy Graph a Vertex do samostatného modulu.

### 2. Načtětě obě třídy do toho notebooku, tak abyste s nimi mohli pracovat.

**Tip**: Když pracujete s jupyter notebooky, může se vám často může stát, že chcete používat tentýž kód ve více noteboocích. To je přesně čas pro přesunutí kódu do samostatného modulu a jeho import do notebooku. Někdy ale váš kód nemusí být ještě úplně stabilní a můžete ho upravovat. Jenomže pokud něco změníte v modulu, jupyter to nezaregistruje. Aby se vám změny zpropagovaly, musíte restartovat kernel, což je poněkud otravné. Naštěstí se tohle dá vyřešit pomocí tzv. autoreloadingu. Pokud přidáte na začátek notebooku následující 2 řádky:

```
%load_ext autoreload
%autoreload 2
```

všechny moduly budou znovu načteny při spuštění jakéhokoli kódu. Tedy nové změny s hned uplatní.
 

In [1]:
# TODO: Implement your solution.
%load_ext autoreload
%autoreload 2

In [11]:
from graphs.Vertex import Vertex
from graphs.Graph import Graph

v1 = Vertex()
print(v1)

Vrchol číslo 7 má sousedy []


## Dědičnost

### 3. Vytvořte novou třídu pro uzly s atributy. Hodnota atributu se pak bude vypisovat ve výpisu objektu. Ostatní funkcionalita uzlu zůstane stejná.

**Tip**: Zjevně příklad na dědičnost.

In [3]:
# TODO: Implement your solution.
class AttributeVertex(vertex):
    
    def __init__(self, attribute):
        super().__init__()
        self.attributes = attribute
    
    def __str__(self):
         return 'Vrchol číslo ' + str(self.id) + ' má hodnotu `' + str(self.attribute) + '` a sousedy ' + str(sorted(self.neighbours))

In [5]:
vv = AttributeVertex('Foo')
print(vv)

Vrchol číslo 2 má hodnotu `Foo` a sousedy []


## ABC

### Bonus: Naimplementujte rozhraní grafu pomocí abstraktní třídy. Abstraktní graf bude definovat abstraktní metody: `connect_vertices`, `edges`, `__str__`. Graf, který už máme bude jedna jeho implementace (například `OrdinaryGraph`). Další implementace pak může třeba být graf, který bude mít hrany ohodnocené atributy. Tyto atributy se tedy budou ukládat přímo v grafu.

In [17]:
 

class AbstractGraph(ABC):
    
    id = 0
    
    def __init__(self, *vertices):
        self.id = self.id_next()
        self.vertices = set(vertices)
        
    @classmethod
    def id_next(cls):
        cls.id += 1
        return cls.id
    
    def add_vertices(self, *vertices):
        self.vertices.update(vertices)
        
    def remove_vertices(self, *vertices):
        for vertex in vertices:
            vertex.remove_neighbours(*vertex.neighbours)
            self.vertices.discard(vertex)
            
    @property
    def vertex_number(self):
        return len(self.vertices)
    
        
    @abstractmethod
    def connect_vertices(self, vertex1, vertex2):
        raise NotImplementedError()
        
    @property
    @abstractmethod
    def edges(self):
        raise NotImplementedError()
        
    @abstractmethod
    def __str__(self):
        raise NotImplementedError()

In [18]:
g = AbstractGraph()


TypeError: Can't instantiate abstract class AbstractGraph with abstract methods __str__, connect_vertices, edges

In [19]:
# TODO: Implement your solution.
class OrdinaryGraph(AbstractGraph):
    
    def connect_vertices(self, vertex1, vertex2):
        vertex1.add_neighbours(vertex2)
   
    @property
    def edges(self):
        edges = set()
        for vertex in self.vertices:
            for neighbour in vertex.neighbours:
                edges.add( frozenset((vertex.id, neighbour.id)) )
        return edges
    
    @property
    def export_graph(self):
        s = str(self.vertex_number) + ';'
        s += ','.join( '-'.join(str(idx) for idx in edge) for edge in self.edges )
        return s
    
    def import_graph(self, xs):
        vertex_number, edges = xs.split(';')
        # vertices
        vertices = {str(i+1):Vertex() for i in range(int(vertex_number))}
        self.add_vertices(*[vertex for idx, vertex in vertices.items()])
        # edges
        edges = edges.split(',')
        for edge in edges:
            f, t = edge.split('-')
            self.connect_vertices(vertices[f], vertices[t])
            
    def __str__(self):
        return 'Graf číslo ' + str(self.id) + ' obsahuje vrcholy ' + str(sorted(self.vertices)) + ' spojené hranami ' + str(self.edges)
    

In [9]:
v1 = Vertex()
v2 = Vertex()
v3 = Vertex()

og = OrdinaryGraph(v1, v2, v3)
og.connect_vertices(v1, v2)
og.connect_vertices(v1, v3)
print(og)

Graf číslo 1 obsahuje vrcholy [2, 3, 4] spojené hranami {frozenset({2, 4}), frozenset({2, 3})}


In [10]:
# TODO: Implement your solution.
class EdgeAttributeGraph(AbstractGraph):
    
    def __init__(self, *vertices):
        super().__init__(vertices)
        self.edges_ = {}
        
    def connect_vertices_with_attribute(self, vertex1, vertex2, attribute):
        vertex1.add_neighbours(vertex2)
        self.edges_[(vertex1.id, vertex2.id)] = attribute

    def connect_vertices(self, vertex1, vertex2):
        self.connect_vertices_with_attribute(vertex1, vertex2, None)
        
    def remove_vertices(self, *vertices):
        for e in self.edges_.keys():
            for v in vertices:
                if e[0] == v.id or e[1] == v.id:
                    del self.edges_[e]
                
        super().remove_vertices(vertices)
    
    @property
    def edges(self):
        return self.edges_.items()
    
    def __str__(self):
        return 'Graf číslo ' + str(self.id) + ' obsahuje vrcholy ' + str(sorted(self.vertices)) + ' spojené hranami ' + str(self.edges)
    

In [11]:
v1 = Vertex()
v2 = Vertex()
v3 = Vertex()

eag = EdgeAttributeGraph(v1, v2, v3)
eag.connect_vertices(v1, v2)
eag.connect_vertices_with_attribute(v1, v3, 'My edge')
print(eag)

Graf číslo 1 obsahuje vrcholy [(5, 6, 7)] spojené hranami dict_items([((5, 6), None), ((5, 7), 'My edge')])
